In [15]:
# pip install -e git+https://github.com/tomquirk/realestate-com-au-api.git#egg=realestate_com_au_api


In [1]:
from realestate_com_au import RealestateComAu

api = RealestateComAu()

# Get property listings
# listings = api.search(locations=["seventeen seventy, qld 4677"], channel="buy", keywords=["tenant"], exclude_keywords=["pool"])



In [2]:
import pandas as pd

# https://www.matthewproctor.com/australian_postcodes
df_postcode = pd.read_excel (r'/Users/joezhou/Downloads/Realestate/australian_postcodes.xlsx', sheet_name='Sheet1')

# sample test suburb
# df_postcode_subset = df_postcode.loc[(df_postcode['Locality']=='LIDCOMBE') & (df_postcode['Postcode']==2141) ]

# df_postcode_subset["VAR_Find"] = '["'+df_postcode_subset['Postcode'].apply(str)+","+df_postcode_subset['Locality']+'"]'
# list_postcode = list(df_postcode_subset['VAR_Find'].dropna())


In [ ]:
# listings_rent = api.search(locations=["ACT, NT, SA, WA, NSW, QLD, VIC, TAS"], channel="rent")
# listings_all = api.search(locations=["ACT, NT, SA, WA, NSW, QLD, VIC, TAS"])

# last run for rent was 7 mins
#note, output caps out at 2,000 records

# Find_Text = ["2141,LIDCOMBE"]
# df_out = api.search(locations=Find_Text,channel="rent")
# 870 search records with single quotes

In [3]:
# try a loop approach by suburb
#note: code below doesn't work for some reason, it returns an arbitary list of values

DF_RENT = pd.DataFrame()
DF_ALL = pd.DataFrame()

def DWN_INF(Find_Text):    
    global DF_RENT
    try:
        df_imp_rent = api.search(locations=Find_Text, channel="rent")
        df_imp_rent = pd.DataFrame(df_imp_rent)
        
        from datetime import datetime as dt
        dte = dt.now().strftime('%Y%m%d')
        fname = "/Users/joezhou/Downloads/loop_test{}.xlsx".format(dte)
        df_imp_rent.to_excel(fname)

        df_imp_all = api.search(locations=Find_Text)
        df_imp_all = pd.DataFrame(df_imp_all)

        DF_RENT = DF_RENT.append(df_imp_rent, ignore_index=True) 
        DF_ALL = DF_ALL.append(df_imp_all, ignore_index=True) 

        print("found for:",Find_Text)

    except Exception:
        print("Error with:",Find_Text)
        pass    
    return()

# Download all
# for Find_Var in list_postcode:
#    DWN_INF(Find_Var)


In [ ]:
DF_RENT = DF_RENT.drop_duplicates()


In [2]:
# for Find_Var in list_postcode:
    # df_imp_rent = api.search(locations=Find_Var, channel="rent")

# DF_RENT.to_pickle("/Users/joezhou/Downloads/Realestate/R_ALL_RENT.pkl")

import pandas as pd
I_INFO = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_ALL_RENT.pkl') 


from datetime import datetime as dt
dte = dt.now().strftime('%Y%m%d')
fname = "/Users/joezhou/Downloads/test_extract_rent_AUS{}.xlsx".format(dte)
I_INFO.to_excel(fname)


In [9]:
df_out = pd.DataFrame(df_out)

from datetime import datetime as dt
dte = dt.now().strftime('%Y%m%d')
fname = "/Users/joezhou/Downloads/test_extract_rent_AUS{}.xlsx".format(dte)
DF_RENT.to_excel(fname)


/Users/joezhou/opt/anaconda3/lib/python3.8/site-packages/xlsxwriter/worksheet.py:941: UserWarning: Ignoring URL 'https://www.2apply.com.au/agency?n=HaymanPartners&form=0%3cbr/%3e%3cbr/%3eIn%20a%20quiet%20cul-de-sac%20with%20uninterrupted%20leafy%20views%20straight%20down%20the%20Curtin%20green%20belt,%20also%20within%20walking%20distance%20to%20sought-after%20schools,%20the%20Curtin%20shops%20and%20the%20Woden%20Shopping%20precinct,%20this%20four-bedroom%20home%20provides%20the%20ultimate%20in%20family%20convenience.%3cbr/%3e%3cbr/%3eA%20decked%20front%20patio%20offers%20the%20prime%20position%20to%20sit%20and%20enjoy%20the%20views%20and%20keep%20a%20watchful%20eye%20on%20children%20playing%20at%20the%20nearby%20playground%20reserve.%20There's%20also%20additional%20entertaining%20space%20in%20the%20rear%20yard's%20paved%20courtyard%20conveniently%20placed%20off%20the%20kitchen/living%20area.%3cbr/%3e%3cbr/%3eFEATURES%3cbr/%3e•%20Single-level%20family%20brick%20home%20at%20the%20end%20o

In [ ]:


# RAW_DATA = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_REALESTATE_ALL.pkl') 

RAW_DATA_SUBSET = DF_ALL.drop(columns=['images', 'images_floorplans', 'listers'])

# summarise by postcode

RAW_DATA_SUM = RAW_DATA_SUBSET.groupby("postcode")["price"].median()
RAW_DATA_SUM = pd.DataFrame(RAW_DATA_SUM)
RAW_DATA_SUM.rename(columns={'price':'price_BUY'}, inplace = True)


In [ ]:
# RAW_DATA_RENT = pd.read_pickle ('/Users/joezhou/Downloads/Realestate/R_REALESTATE_RENT.pkl') 

RAW_DATA_RENT_SUBSET = DF_RENT.drop(columns=['images', 'images_floorplans', 'listers'])

# summarise by postcode

RAW_DATA_RENT_SUM = RAW_DATA_RENT_SUBSET.groupby("postcode")["price"].median()
RAW_DATA_RENT_SUM = pd.DataFrame(RAW_DATA_RENT_SUM)
RAW_DATA_RENT_SUM.rename(columns={"price": "price_RENT"}, inplace = True)


In [ ]:
df_comp = RAW_DATA_SUM.merge(RAW_DATA_RENT_SUM,how='left',on='postcode')

df_comp['YEARLY_RENT']=df_comp['price_RENT']*52
df_comp['RENT_YIELD']=df_comp['YEARLY_RENT']/df_comp['price_BUY']

# 7303

INFO:numexpr.utils:NumExpr defaulting to 4 threads.
